In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, convert_to_messages
from os.path import dirname, join
from dotenv import load_dotenv
from typing import Literal
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.types import Command
from operator import add
import pandas as pd
import getpass
import os

In [ ]:
# Set your OpenAI API key
load_dotenv("/Users/mastorga/Documents/BTE-LLM/.env")

if not os.environ.get("OPENAI_API_KEY"): #field to ask for OpenAI API key
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter OpenAI API Key: ")

In [ ]:
from Agent import BTEx

In [ ]:
# Create an LLM-based agent
llm = ChatOpenAI(temperature=0, model="gpt-4o")  # Change model if needed

In [ ]:
def get_llm_response(question):
    return llm.invoke(question).content

In [ ]:
drugDiseaseDMDB = pd.read_csv('/Users/mastorga/Documents/BTE-LLM/Prototype/data/DMDB_go_bp_filtered_jjoy_05_08_2025.csv')

In [ ]:
drugDiseaseSet = drugDiseaseDMDB.sample(25)

print(drugDiseaseSet)

In [ ]:
# Question only + w/o ID (w/o ID)

drugDiseaseSet['question w/o ID'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " by targeting " + drugDiseaseSet['bp_name'] + "?"

In [ ]:
# Question + addtl directions (top 10) + w/o ID (w/o ID - top 10)

drugDiseaseSet['question w/o ID - top 10'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " by targeting " + drugDiseaseSet['bp_name'] + "? Enumerate 10 drugs and do not include anything else in your response."

In [ ]:
# Question + addtl directions (top 20) + w/o ID (w/o ID - top 20)

drugDiseaseSet['question w/o ID - top 20'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " by targeting " + drugDiseaseSet['bp_name'] + "? Enumerate 20 drugs and do not include anything else in your response."

In [ ]:
# Question + addtl directions (top 30) + w/o ID (w/o ID - top 30)

drugDiseaseSet['question w/o ID - top 30'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " by targeting " + drugDiseaseSet['bp_name'] + "? Enumerate 30 drugs and do not include anything else in your response."

In [ ]:
# LLM answer (w/o ID)

drugDiseaseSet['gpt4.0_response w/o ID'] = drugDiseaseSet["question w/o ID"].apply(get_llm_response)

In [ ]:
# LLM answer (w/o ID - top 10)

drugDiseaseSet['gpt4.0_response w/o ID - top 10'] = drugDiseaseSet["question w/o ID - top 10"].apply(get_llm_response)

In [ ]:
# LLM answer (w/o ID - top 20)

drugDiseaseSet['gpt4.0_response w/o ID - top 20'] = drugDiseaseSet["question w/o ID - top 20"].apply(get_llm_response)

In [ ]:
# LLM answer (w/o ID - top 30)

drugDiseaseSet['gpt4.0_response w/o ID - top 30'] = drugDiseaseSet["question w/o ID - top 30"].apply(get_llm_response)

In [ ]:

drugDiseaseSet.to_excel('/Users/mastorga/Documents/BTE-LLM/Prototype/logs/drug <- diseasebp/08.05.25_inputPromptTuning_25q.xlsx')

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/o ID)

drugDiseaseSet["BTEx_response (w/o ID) (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/o ID"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/o ID - top 10)

drugDiseaseSet["BTEx_response (w/o ID top 10) (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/o ID - top 10"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/o ID - top 20)

drugDiseaseSet["BTEx_response (w/o ID top 20) (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/o ID - top 20"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/o ID - top 30)

drugDiseaseSet["BTEx_response (w/o ID top 30) (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/o ID - top 30"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# Question + addtl directions (top 5) + w/o ID (w/o ID - top 5)

drugDiseaseSet['question w/o ID - top 5'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " by targeting " + drugDiseaseSet['bp_name'] + "? Enumerate 5 drugs and do not include anything else in your response."

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/o ID - top 5)

drugDiseaseSet["BTEx_response (w/o ID top 5) (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/o ID - top 5"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# LLM answer (w/o ID - top 10)

drugDiseaseSet['gpt4.0_response w/o ID - top 5'] = drugDiseaseSet["question w/o ID - top 5"].apply(get_llm_response)

In [ ]:
# Save responses

drugDiseaseSet.to_excel('/Users/mastorga/Documents/BTE-LLM/Prototype/logs/drug <- diseasebp/RAGresponse2_08.05.25_inputPromptTuning_25q.xlsx')

In [ ]:
# Question only + w/ ID

drugDiseaseSet['question w/ ID'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " (ID: " + drugDiseaseSet['disease'] + ") by targeting " + drugDiseaseSet['bp_name'] + " (ID: " + drugDiseaseSet['bp'] + ")?"

In [ ]:
# LLM answer (w/ ID)

drugDiseaseSet['gpt4o_response w/ ID'] = drugDiseaseSet["question w/ ID"].apply(get_llm_response)

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/ ID)

drugDiseaseSet["BTEx_response (w/ ID) (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/ ID"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# Save responses

drugDiseaseSet.to_excel('/Users/mastorga/Documents/BTE-LLM/Prototype/logs/drug <- diseasebp/RAGresponse3_08.05.25_inputPromptTuning_25q.xlsx')